In [ ]:
# ARC-AGI Data Exploration

# This notebook explores the ARC-AGI dataset to understand:
# - Task structure and patterns
# - Grid size distributions
# - Color usage patterns
# - Training vs test characteristics
# - Visualization of example tasks

import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import pandas as pd
from typing import Dict, List, Tuple

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")

# ARC color mapping for visualization
ARC_COLORS = {
    0: '#000000',  # black
    1: '#1E90FF',  # blue  
    2: '#FF6347',  # red
    3: '#32CD32',  # green
    4: '#FFD700',  # yellow
    5: '#A9A9A9',  # gray
    6: '#FF1493',  # magenta
    7: '#FF8C00',  # orange
    8: '#87CEEB',  # sky blue
    9: '#8B4513'   # brown
}

def plot_grid(grid, title="", ax=None):
    """Plot an ARC grid with proper colors"""
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    
    grid = np.array(grid)
    
    # Create color matrix
    colored_grid = np.zeros((*grid.shape, 3))
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            color = ARC_COLORS[grid[i, j]]
            colored_grid[i, j] = [int(color[1:3], 16)/255, int(color[3:5], 16)/255, int(color[5:7], 16)/255]
    
    ax.imshow(colored_grid, aspect='equal')
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Add grid lines
    for i in range(grid.shape[0] + 1):
        ax.axhline(i - 0.5, color='black', linewidth=0.5)
    for j in range(grid.shape[1] + 1):
        ax.axvline(j - 0.5, color='black', linewidth=0.5)

def load_arc_data(split='training'):
    """Load ARC data from JSON files"""
    try:
        if split == 'training':
            with open('/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json', 'r') as f:
                challenges = json.load(f)
            with open('/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json', 'r') as f:
                solutions = json.load(f)
        elif split == 'evaluation':
            with open('/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json', 'r') as f:
                challenges = json.load(f)
            with open('/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json', 'r') as f:
                solutions = json.load(f)
        else:
            with open('/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json', 'r') as f:
                challenges = json.load(f)
            solutions = None
        
        print(f"✓ Loaded {len(challenges)} {split} challenges")
        return challenges, solutions
        
    except FileNotFoundError as e:
        print(f"✗ Could not load {split} data: {e}")
        return None, None

print("Data loading function defined")

# Load training data for exploration
print("Loading ARC training data...")
training_challenges, training_solutions = load_arc_data('training')

if training_challenges is None:
    print("No training data found. Using sample data for demonstration.")
    # Create sample data structure
    training_challenges = {
        'sample_task_1': {
            'train': [{
                'input': [[0, 1], [1, 0]],
                'output': [[1, 0], [0, 1]]
            }],
            'test': [{
                'input': [[0, 2], [2, 0]]
            }]
        }
    }
    training_solutions = {
        'sample_task_1': [[[2, 0], [0, 2]]]
    }

print(f"Working with {len(training_challenges)} tasks")

# Analyze dataset statistics
print("Analyzing dataset statistics...")

stats = {
    'total_tasks': len(training_challenges),
    'train_examples_per_task': [],
    'test_examples_per_task': [],
    'grid_sizes': [],
    'colors_used': Counter(),
    'unique_colors_per_grid': [],
    'grid_transformations': [],
    'input_output_size_relations': []
}

for task_id, task_data in training_challenges.items():
    # Count examples
    stats['train_examples_per_task'].append(len(task_data['train']))
    stats['test_examples_per_task'].append(len(task_data['test']))
    
    # Analyze training examples
    for example in task_data['train']:
        input_grid = np.array(example['input'])
        output_grid = np.array(example['output'])
        
        # Grid sizes
        stats['grid_sizes'].append(('input', input_grid.shape))
        stats['grid_sizes'].append(('output', output_grid.shape))
        
        # Colors
        input_colors = set(input_grid.flatten())
        output_colors = set(output_grid.flatten())
        
        for color in input_colors:
            stats['colors_used'][color] += 1
        for color in output_colors:
            stats['colors_used'][color] += 1
            
        stats['unique_colors_per_grid'].append(len(input_colors))
        stats['unique_colors_per_grid'].append(len(output_colors))
        
        # Size relationships
        if input_grid.shape == output_grid.shape:
            stats['input_output_size_relations'].append('same')
        elif input_grid.size > output_grid.size:
            stats['input_output_size_relations'].append('shrink')
        else:
            stats['input_output_size_relations'].append('grow')
    
    # Analyze test examples  
    for example in task_data['test']:
        input_grid = np.array(example['input'])
        stats['grid_sizes'].append(('test_input', input_grid.shape))
        
        input_colors = set(input_grid.flatten())
        for color in input_colors:
            stats['colors_used'][color] += 1
        stats['unique_colors_per_grid'].append(len(input_colors))

print("Analysis complete!")

# Display basic statistics
print("📊 DATASET OVERVIEW")
print("=" * 50)
print(f"Total tasks: {stats['total_tasks']}")
print(f"Train examples per task: {np.mean(stats['train_examples_per_task']):.1f} ± {np.std(stats['train_examples_per_task']):.1f}")
print(f"Test examples per task: {np.mean(stats['test_examples_per_task']):.1f} ± {np.std(stats['test_examples_per_task']):.1f}")
print(f"Unique colors per grid: {np.mean(stats['unique_colors_per_grid']):.1f} ± {np.std(stats['unique_colors_per_grid']):.1f}")

# Grid size analysis
grid_sizes = [size for _, size in stats['grid_sizes']]
grid_areas = [size[0] * size[1] for size in grid_sizes]
grid_widths = [size[1] for size in grid_sizes]
grid_heights = [size[0] for size in grid_sizes]

print(f"\n📐 GRID DIMENSIONS")
print(f"Width range: {min(grid_widths)} - {max(grid_widths)} (avg: {np.mean(grid_widths):.1f})")
print(f"Height range: {min(grid_heights)} - {max(grid_heights)} (avg: {np.mean(grid_heights):.1f})")
print(f"Area range: {min(grid_areas)} - {max(grid_areas)} (avg: {np.mean(grid_areas):.1f})")

# Size relationships
size_rel_counts = Counter(stats['input_output_size_relations'])
print(f"\n🔄 INPUT-OUTPUT SIZE RELATIONSHIPS")
for rel, count in size_rel_counts.items():
    percentage = count / len(stats['input_output_size_relations']) * 100
    print(f"{rel}: {count} ({percentage:.1f}%)")

# Create comprehensive visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('ARC-AGI Dataset Analysis', fontsize=16, fontweight='bold')

# 1. Examples per task distribution
axes[0, 0].hist(stats['train_examples_per_task'], bins=range(1, max(stats['train_examples_per_task'])+2), 
                alpha=0.7, label='Train', edgecolor='black')
axes[0, 0].hist(stats['test_examples_per_task'], bins=range(1, max(stats['test_examples_per_task'])+2), 
                alpha=0.7, label='Test', edgecolor='black')
axes[0, 0].set_xlabel('Examples per Task')
axes[0, 0].set_ylabel('Number of Tasks')
axes[0, 0].set_title('Examples per Task Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Grid size distribution
axes[0, 1].scatter(grid_widths, grid_heights, alpha=0.6, s=20)
axes[0, 1].set_xlabel('Grid Width')
axes[0, 1].set_ylabel('Grid Height')
axes[0, 1].set_title('Grid Size Distribution')
axes[0, 1].grid(True, alpha=0.3)

# 3. Color usage frequency
colors = list(stats['colors_used'].keys())
counts = list(stats['colors_used'].values())
color_names = ['Black', 'Blue', 'Red', 'Green', 'Yellow', 'Gray', 'Magenta', 'Orange', 'Sky Blue', 'Brown']
bars = axes[0, 2].bar(range(len(colors)), counts, 
                      color=[ARC_COLORS[c] for c in colors], 
                      edgecolor='black', linewidth=0.5)
axes[0, 2].set_xlabel('ARC Colors')
axes[0, 2].set_ylabel('Usage Frequency')
axes[0, 2].set_title('Color Usage Distribution')
axes[0, 2].set_xticks(range(len(colors)))
axes[0, 2].set_xticklabels([f'{c}' for c in colors], rotation=45)
axes[0, 2].grid(True, alpha=0.3)

# 4. Unique colors per grid
axes[1, 0].hist(stats['unique_colors_per_grid'], bins=range(1, max(stats['unique_colors_per_grid'])+2), 
                alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Unique Colors per Grid')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Color Diversity Distribution')
axes[1, 0].grid(True, alpha=0.3)

# 5. Grid area distribution
axes[1, 1].hist(grid_areas, bins=30, alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Grid Area (pixels)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Grid Area Distribution')
axes[1, 1].grid(True, alpha=0.3)

# 6. Size relationship pie chart
if stats['input_output_size_relations']:
    size_rel_counts = Counter(stats['input_output_size_relations'])
    labels = list(size_rel_counts.keys())
    sizes = list(size_rel_counts.values())
    axes[1, 2].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    axes[1, 2].set_title('Input-Output Size Relationships')
else:
    axes[1, 2].text(0.5, 0.5, 'No size\nrelationship\ndata', 
                    ha='center', va='center', transform=axes[1, 2].transAxes)
    axes[1, 2].set_title('Input-Output Size Relationships')

plt.tight_layout()
plt.show()

# Visualize example tasks
def visualize_task(task_id, task_data, solutions=None, max_examples=3):
    """Visualize a complete ARC task with train/test examples"""
    n_train = min(len(task_data['train']), max_examples)
    n_test = min(len(task_data['test']), max_examples)
    
    fig, axes = plt.subplots(n_train + n_test, 3, figsize=(12, 4*(n_train + n_test)))
    if (n_train + n_test) == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle(f'Task: {task_id}', fontsize=14, fontweight='bold')
    
    row = 0
    
    # Training examples
    for i in range(n_train):
        example = task_data['train'][i]
        
        plot_grid(example['input'], f'Train {i+1} - Input', axes[row, 0])
        plot_grid(example['output'], f'Train {i+1} - Output', axes[row, 1])
        
        # Transformation analysis
        input_shape = np.array(example['input']).shape
        output_shape = np.array(example['output']).shape
        input_colors = len(set(np.array(example['input']).flatten()))
        output_colors = len(set(np.array(example['output']).flatten()))
        
        transform_text = f"Shape: {input_shape} → {output_shape}\n"
        transform_text += f"Colors: {input_colors} → {output_colors}\n"
        
        if input_shape == output_shape:
            transform_text += "Type: Same size"
        elif np.prod(input_shape) > np.prod(output_shape):
            transform_text += "Type: Shrinking"
        else:
            transform_text += "Type: Growing"
        
        axes[row, 2].text(0.1, 0.5, transform_text, transform=axes[row, 2].transAxes, 
                         fontsize=10, verticalalignment='center',
                         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
        axes[row, 2].set_xlim(0, 1)
        axes[row, 2].set_ylim(0, 1)
        axes[row, 2].set_xticks([])
        axes[row, 2].set_yticks([])
        axes[row, 2].set_title(f'Train {i+1} - Analysis')
        
        row += 1
    
    # Test examples
    for i in range(n_test):
        example = task_data['test'][i]
        
        plot_grid(example['input'], f'Test {i+1} - Input', axes[row, 0])
        
        # Show solution if available
        if solutions and task_id in solutions and i < len(solutions[task_id]):
            plot_grid(solutions[task_id][i], f'Test {i+1} - Solution', axes[row, 1])
        else:
            axes[row, 1].text(0.5, 0.5, '?', ha='center', va='center', 
                             transform=axes[row, 1].transAxes, fontsize=48,
                             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray"))
            axes[row, 1].set_xlim(0, 1)
            axes[row, 1].set_ylim(0, 1)
            axes[row, 1].set_xticks([])
            axes[row, 1].set_yticks([])
            axes[row, 1].set_title(f'Test {i+1} - Unknown')
        
        # Test analysis
        input_shape = np.array(example['input']).shape
        input_colors = len(set(np.array(example['input']).flatten()))
        
        test_text = f"Input Shape: {input_shape}\n"
        test_text += f"Input Colors: {input_colors}\n"
        test_text += "Task: Apply learned rule"
        
        axes[row, 2].text(0.1, 0.5, test_text, transform=axes[row, 2].transAxes, 
                         fontsize=10, verticalalignment='center',
                         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.7))
        axes[row, 2].set_xlim(0, 1)
        axes[row, 2].set_ylim(0, 1)
        axes[row, 2].set_xticks([])
        axes[row, 2].set_yticks([])
        axes[row, 2].set_title(f'Test {i+1} - Analysis')
        
        row += 1
    
    plt.tight_layout()
    plt.show()

# Show a few example tasks
print("🎯 EXAMPLE TASK VISUALIZATIONS")
print("=" * 50)

task_ids = list(training_challenges.keys())[:3]  # Show first 3 tasks
for task_id in task_ids:
    print(f"\nVisualizing task: {task_id}")
    visualize_task(task_id, training_challenges[task_id], training_solutions)

# Advanced pattern analysis
print("🔍 ADVANCED PATTERN ANALYSIS")
print("=" * 50)

def analyze_transformations(challenges):
    """Analyze common transformation patterns"""
    patterns = {
        'identity': 0,  # Input == Output
        'rotation': 0,
        'reflection': 0,
        'color_change': 0,
        'scaling': 0,
        'complex': 0
    }
    
    for task_id, task_data in challenges.items():
        for example in task_data['train']:
            input_grid = np.array(example['input'])
            output_grid = np.array(example['output'])
            
            if np.array_equal(input_grid, output_grid):
                patterns['identity'] += 1
            elif input_grid.shape != output_grid.shape:
                patterns['scaling'] += 1
            elif np.array_equal(input_grid, np.rot90(output_grid, k=1)) or \
                 np.array_equal(input_grid, np.rot90(output_grid, k=2)) or \
                 np.array_equal(input_grid, np.rot90(output_grid, k=3)):
                patterns['rotation'] += 1
            elif np.array_equal(input_grid, np.fliplr(output_grid)) or \
                 np.array_equal(input_grid, np.flipud(output_grid)):
                patterns['reflection'] += 1
            elif input_grid.shape == output_grid.shape and \
                 not np.array_equal((input_grid > 0), (output_grid > 0)):
                patterns['color_change'] += 1
            else:
                patterns['complex'] += 1
    
    return patterns

transformation_patterns = analyze_transformations(training_challenges)

print("Transformation patterns found:")
total_examples = sum(transformation_patterns.values())
for pattern, count in transformation_patterns.items():
    percentage = count / total_examples * 100 if total_examples > 0 else 0
    print(f"{pattern}: {count} ({percentage:.1f}%)")

# Visualize transformation patterns
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
patterns = list(transformation_patterns.keys())
counts = list(transformation_patterns.values())

bars = ax.bar(patterns, counts, color='skyblue', edgecolor='navy', alpha=0.7)
ax.set_xlabel('Transformation Type')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Transformation Patterns')
ax.grid(True, alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{count}', ha='center', va='bottom')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Generate exploration summary report
print("\n📋 EXPLORATION SUMMARY REPORT")
print("=" * 60)

report = f"""
ARC-AGI Dataset Exploration Summary
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

📊 DATASET OVERVIEW:
• Total tasks analyzed: {stats['total_tasks']}
• Average training examples per task: {np.mean(stats['train_examples_per_task']):.1f}
• Average test examples per task: {np.mean(stats['test_examples_per_task']):.1f}

📐 GRID CHARACTERISTICS:
• Grid width range: {min(grid_widths)} - {max(grid_widths)} (avg: {np.mean(grid_widths):.1f})
• Grid height range: {min(grid_heights)} - {max(grid_heights)} (avg: {np.mean(grid_heights):.1f})
• Grid area range: {min(grid_areas)} - {max(grid_areas)} (avg: {np.mean(grid_areas):.1f})

🎨 COLOR USAGE:
• Most common color: {max(stats['colors_used'], key=stats['colors_used'].get)} (used {max(stats['colors_used'].values())} times)
• Least common color: {min(stats['colors_used'], key=stats['colors_used'].get)} (used {min(stats['colors_used'].values())} times)
• Average unique colors per grid: {np.mean(stats['unique_colors_per_grid']):.1f}

🔄 TRANSFORMATIONS:
"""

for pattern, count in transformation_patterns.items():
    percentage = count / total_examples * 100 if total_examples > 0 else 0
    report += f"• {pattern.title()}: {count} examples ({percentage:.1f}%)\n"

size_rel_counts = Counter(stats['input_output_size_relations'])
report += "\n🔍 SIZE RELATIONSHIPS:\n"
for rel, count in size_rel_counts.items():
    percentage = count / len(stats['input_output_size_relations']) * 100
    report += f"• {rel.title()}: {count} examples ({percentage:.1f}%)\n"

report += f"""
💡 KEY INSIGHTS:
• Grid sizes vary significantly, requiring flexible architectures
• Color usage is uneven - black (0) typically most common
• Most transformations are complex, suggesting need for sophisticated models
• Size relationships are important - many tasks maintain grid dimensions

🎯 IMPLICATIONS FOR MODEL DESIGN:
• Multi-scale processing needed for variable grid sizes
• Color-aware representations important
• Spatial reasoning capabilities essential
• Pattern recognition across different scales required
"""

print(report)

# Save report to file
try:
    import os
    os.makedirs('../reports', exist_ok=True)
    with open('../reports/exploration_report.txt', 'w') as f:
        f.write(report)
    print("\n💾 Report saved to ../reports/exploration_report.txt")
except:
    print("\n⚠ Could not save report to file (directory may not exist)")

print("\n🎉 Exploration complete!")